In [ ]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

# A Backtracking Solver for CSPs

## Utility Functions

The module `ast` provides a parser for Python expressions.

In [ ]:
import ast

The function `collect_variables(expr)` takes a string `expr` that can be interpreted as a Python expression as input and collects all variables occurring in `expr`.  It takes care to remove the function symbols from the names returned by `collect_variables`.

In [ ]:
def collect_variables(expression): 
    tree = ast.parse(expression)
    return { node.id for node in ast.walk(tree) 
                     if  isinstance(node, ast.Name) 
                     if  node.id not in dir(__builtins__)
           }

Below, `dir(__builtins__)` returns a list containing all predefined variables, functions, and classes. 

In [ ]:
dir(__builtins__)

In [ ]:
collect_variables('abs(x - y) + abs(z1 - z2)')

## The Backtracking Solver with Animation

The procedure `solve(P)` takes a *constraint satisfaction problem* 
`P` as input.  Here `P` is a triple of the form 
$$ \mathcal{P} = \langle \mathtt{Variables}, \mathtt{Values}, \mathtt{Constraints} \rangle $$
where 
- `Variables` is a set of strings which serve as *variables*,
- `Values` is a set of *values* that can be assigned 
  to the variables in the set `Variables`.
- `Constraints` is a set of formulas from first order logic.  
  
  
The main purpose of the function `solve` is to convert the CSP `P` into an 
*augmented CSP* where every constraint $f$ is annotated with
the variables ocurring in $f$.  This annotated CSP is then solved using the function
`backtrack_search`.

In [ ]:
n = 0

In [ ]:
def solve(P):
    global n
    n = 0
    Variables, Values, Constraints = P
    csp = (Variables, Values, [(f, collect_variables(f)) for f in Constraints])
    Solution = backtrack_search({}, csp)
    print(f'tested {n} partial assignments')
    return Solution

The function `backtrack_search` takes two arguments:
- `Assignment` is a partial variable assignment that is represented as a dictionary.  Initially, this assignment will be the empty dictionary.  Every recursive call of `backtrack_search` adds the assignment of one variable to the given assignment.  The important invariant of recursive calls of `backtrack_search` is that `Assignment` is *consistent*, i.e. all constraints $f$ that contain only variables from the set $\mathtt{dom}(\mathtt{Assignment})$ are satisfied.
- `P` is an *augmented constraint satisfaction problem*, 
   i.e. `P` is a triple of the form 
   $$ \mathcal{P} = \langle \mathtt{Vars}, \mathtt{Values}, \mathtt{Constraints} \rangle $$
    where 
    - $\mathtt{Vars}$ is a set of strings which serve as *variables*,
    - $\mathtt{Values}$ is a set of *values* that can be assigned 
      to the variables in $\mathtt{Vars}$.
    - $\mathtt{Constraints}$ is a set of pairs of the form $(f, V)$ where $f$ is a Boolean Python 
      expression, while $V$ is the set of variables occuring in $f$.
     
The function `backtrack_search` tries to find a solution of `P` by recursively augmenting `Assignment`.

In [ ]:
def backtrack_search(Assignment, P):
    global n
    display(show_solution(Assignment, width="30%"))
    Variables, Values, Constraints = P
    if len(Assignment) == len(Variables):
        return n, Assignment
    var = [x for x in Variables if x not in Assignment][0]
    for value in Values:
        n += 1
        if is_consistent(var, value, Assignment, Constraints):
            NewAss = Assignment.copy()
            NewAss[var] = value
            Solution = backtrack_search(NewAss, P)
            if Solution != None:
                return Solution

The function $\texttt{is_consistent}(\texttt{var}, \texttt{value}, \texttt{Assignment}, \texttt{Constraints})$ takes four arguments:
- `var` is a variable that does not occur in $\texttt{Assignment}$,
- `value` is a value that can be substituted for this variable,
- `Assignment` is a *consistent* partial variable assignment. 
- `Constraints` is a set of pairs of the form $\langle f, V \rangle$ where $f$ is a formula and $V$ is the set of variables occurring in $f$.

This function returns `True` iff the partial variable assignment 
$$\texttt{Assignment} \cup \bigl\{\langle\texttt{var} \mapsto\texttt{value}\rangle\bigr\}$$
is consistent with all the constraints $f$ occurring in `Constraints`.

**Note** that the function `eval` mutates the given variable assignment.  Therefore it is necessary to copy the given `Assignment`.  

In [ ]:
def is_consistent(var, value, Assignment, Constraints):
    NewAssign      = Assignment.copy()
    NewAssign[var] = value
    return all(eval(f, NewAssign) for (f, Vs) in Constraints
                                  if  var in Vs and Vs <= NewAssign.keys()
              )